In [1]:
import pandas as pd
import os
import csv
import re
import logging
import optparse
import re
import spacy
import dedupe
import pickle
import copy
import json
from unidecode import unidecode

In [2]:
sp = spacy.load('en_core_web_sm')

In [ ]:
with open('translations_lookup_all.json') as fin:
    variants = json.load(fin)

with open('langs_dict.json') as fin:
    json.load(fin)
    
def get_type(record):
    name = record['name'].lower()
    
    if pd.isna(record['size']):
        if 'tv' in name:
            return 'tv'
        return 'mobile'
    
    
    flash_keywords = ['usb', 'drive']
    memory_stick_keywords = ['card', 'stick', 'sd', 'microsd', 'hc', 'class', 'speicherkarte'] # Add variants here
    
    is_flash = False
    is_memory = False
    
    for w in flash_keywords:
        if w in name:
            is_flash = True
            break
            
    for w in memory_stick_keywords:
        if w in name:
            is_memory = True
            break
    
    if is_flash:
        return 'flash'
    
    if is_memory:
        return 'stick'
    
    return 'stick'

def convert_numbers_to_strings(df, cols_to_convert, remove_point_zero=True):
    """ Convert number types to strings in a dataframe.
        This is convoluted as need to keep NoneTypes as NoneTypes for what comes next!
        
        Inputs: - df -> dataframe to convert number types
                - cols_to_convert -> list of columns to convert
                - remove_point_zero -> bool to say whether you want '.0' removed from number
        
        Ouputs: - dataframe with converted number types
    """
    new_df = df.copy()
    for col in cols_to_convert:
        if remove_point_zero:
            new_df[col] = new_df[col].apply(lambda x: str(x).replace('.0','')\
                                            if not isinstance(x, type(None)) else x)
        else:
            new_df[col] = new_df[col].apply(lambda x: str(x)\
                                            if not isinstance(x, type(None)) else x)
    return new_df


def preprocess_products_dataset(x4_dev):
    # Clean x4
    # Alpha numeric
    irrelevant_regex = re.compile(r'[^a-z0-9,.\-\s]')
    multispace_regex = re.compile(r'\s\s+') # Why it doesn't work
    x4_dev.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

    for column in x4_dev.columns:
        if column == 'instance_id':
            continue
        x4_dev[column] = x4_dev[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')
        
    x4_dev['product_type'] = x4_dev.apply(get_type, axis=1)
    x4_dev.drop('price', inplace=True, axis=1)
    x4_dev['size'] = x4_dev['size'].str.lower().str.replace(' ', '')
    x4_dev['size'] = x4_dev['size'].where(x4_dev['size'].notnull(), 0)
    
    # Remove unwanted words from the name
    for i in range(len(x4_dev)):
        record = x4_dev.iloc[i]

        name = record['name']

        # remove unnecessary characters
        basic_punct = '-/\*_,:;/()®™' 
        punct_to_space = str.maketrans(basic_punct, ' ' * len(basic_punct))  # map punctuation to space
        name = name.translate(punct_to_space)

        # remove brand
        name = name.replace(record['brand'], '')

        # remove size

        if record.product_type in ['flash', 'stick']:
            name = re.sub('\d\d\d\s?gb', '', name, 6)
            name = re.sub('\d\d\s?gb', '', name, 6)
            name = re.sub('\d\s?gb', '', name, 6)

        tokens = name.split(' ')
        for wd, wdtl in variants.items():
            while wd in tokens:
                tokens.remove(wd)
            for wdt in wdtl:
                while wdt in tokens:
                    tokens.remove(wdt) 

        unneeded_words = ['mmoire', 'speicherkarte', 'flashgeheugenkaart', 'flash', 'stick', 'speed', 'high']
        for w in unneeded_words:
            while w in tokens:
                tokens.remove(w)
        x4_dev.iloc[i]['name'] = ' '.join(tokens)

    for column in x4_dev.columns:
        if column == 'instance_id':
            continue
        x4_dev[column] = x4_dev[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')
        
    return x4_dev


In [19]:
x4 = pd.read_csv('../data/sigmod/X4.csv')
x4_dev = convert_numbers_to_strings(x4, ['price']).copy(deep=True)
x4_dev.set_index('instance_id', inplace=True)
x4_dev = preprocess_products_dataset(x4_dev)

In [20]:
to_dedupe = x4_dev[[
    'name',
    'brand',
    'size',
    'product_type']].copy()

In [21]:
to_dedupe.head()

,name,brand,size,product_type
instance_id,,,,
altosight.com//0,1400x 210mb s professional xqd memriakrtya,lexar,32gb,stick
altosight.com//25,microsdxc uhs 1 u3 memriakrtya adapter srg1uxa,sony,128gb,stick
altosight.com//66,dual drive type c usb 3.0 130 mb s,sandisk,16gb,flash
altosight.com//68,dual drive type c usb 3.0 150 mb s,sandisk,64gb,flash
altosight.com//94,xqd x1400 professional xqd kupon premium 50 n...,lexar,32gb,stick


In [22]:
to_dedupe_dict = to_dedupe.to_dict(orient = 'index')
list(to_dedupe_dict.keys())[0]

'altosight.com//0'

In [23]:
# docs for this are here: 
fields = [{'field' : 'name', 'type' : 'String', 'has_missing' : False},
          {'field' : 'brand', 'type': 'Categorical', 'categories' : list(x4_dev.brand.unique()), 'has_missing' : False}, 
          {'field' : 'size', 'type': 'Exact', 'has_missing' : False},
          {'field' : 'product_type', 'type': 'Categorical', 'categories' : list(x4_dev.product_type.unique()),
           'has_missing' : False}]

In [24]:
# There is a bug later on that requires num_cores to be 1, but we can make use of
# multi-threaded processes in the meantime
deduper = dedupe.Dedupe(fields, num_cores=6)

In [25]:
y = pd.read_csv('../data/sigmod/Y4.csv')

In [26]:
trainig_data = {'match': [], 'distinct': []}

In [27]:
match = y[y.label == 1].to_dict(orient='row')
distinct = y[y.label == 0].to_dict(orient='row')

d:\coding projects\sigmod\venv\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(
d:\coding projects\sigmod\venv\lib\site-packages\pandas\core\frame.py:1549: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [28]:
for m in match:
    trainig_data['match'].append( ( to_dedupe_dict[m['left_instance_id']], to_dedupe_dict[m['right_instance_id']] ) )

In [29]:
len(trainig_data['match'])

4082

In [30]:
for d in distinct:
    trainig_data['distinct'].append( ( to_dedupe_dict[d['left_instance_id']], to_dedupe_dict[d['right_instance_id']] ) )

In [31]:
len(trainig_data['distinct'])

344113

In [32]:
with open('y4_train.json', 'w') as fout:
    json.dump(trainig_data, fout)
    
training_file = 'y4_train.json'
with open(training_file) as tf:
    deduper.prepare_training(to_dedupe_dict, training_file=tf, sample_size=1500, blocked_proportion=0.9)

INFO:dedupe.api:reading training from file
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (wholeFieldPredicate, name)
Exception ignored in: 'lbfgs._lowlevel.call_eval'
Traceback (most recent call last):
  File "d:\coding projects\sigmod\venv\lib\site-packages\rlr\lr.py", line 73, in loss
    g[:] = gradient(x0, X, y, case_weights, alpha)
  File "d:\coding projects\sigmod\venv\lib\site-packages\rlr\lr.py", line 86, in gradient
    grad_c = z0.sum() 
  File "d:\coding projects\sigmod\venv\lib\site-packages\numpy\core\_methods.py", line 47, in _sum
    return umr_sum(a, axis, dtype, out, keepdims, initial, where)
KeyboardInterrupt: 
Exception ignored in: 'lbfgs._lowlevel.call_eval'
Traceback (most recent call last):
  File "d:\coding projects\sigmod\venv\lib\site-packages\rlr\lr.py", line 73, in loss
    g[:] = gradient(x0, X, y, case_weights, alpha)
  File "d:\coding projects\sigmod\venv\lib\site-packages\rlr\lr.py", line 82, in gradient
    z = X.dot(w) 

KeyboardInterrupt: 

KeyboardInterrupt: 

In [33]:
# print('starting active labeling...')
# dedupe.console_label(deduper)

In [34]:
# with open('y_4_augmented.json', 'w') as fout:
#     json.dump(deduper.training_pairs, fout)

In [ ]:
deduper.train(recall=0.9)

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...


In [ ]:
training_file = 'trained_model_x4.json'
settings_file = 'trained_model_x4_settings.json'

with open(training_file, 'w') as tf:
    deduper.write_training(tf)
with open(settings_file, 'wb') as sf:
    deduper.write_settings(sf)

In [ ]:
clustered_dupes = deduper.partition(to_dedupe_dict, 0.5)
print('# duplicate sets', len(clustered_dupes))

In [ ]:
res = []
for el in clustered_dupes:
    for i in range(len(el[0])):
        for j in range(i+1, len(el[0])):
            res.append((el[0][i], el[0][j]))
res_df =pd.DataFrame(res)          
res_df.columns = ['left_instance_id', 'right_instance_id']
res_df.to_csv("output_x4.csv", index=False)

In [ ]:
deduper.predicates

In [ ]:
deduper.partition(trainig_data, 0.5)

In [ ]:
deduper.fields